## Importing Libraries

In [1]:
import os
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
import pandas as pd

c:\Users\surab\OneDrive\Documents\NCSU\Foodchain project\who-eats-whom\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and retreive neo4j variables from .env file

In [6]:
#Storing Neo4j connection details in variables
URI = "neo4j://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "researchgrade"
DB_NAME = "neo4j"

## Initialize the Graph Data Science client

In [9]:
gds = GraphDataScience(
    URI,
    auth=(USER, PASSWORD),
    database=DB_NAME
)

In [10]:

print("Connected to Neo4j GDS server version:", gds.version())

Connected to Neo4j GDS server version: 2.25.0


In [11]:
print("Available GDS algorithms (first few rows):")
print(gds.list().head())

Available GDS algorithms (first few rows):
                                         name  \
0           gds.allShortestPaths.delta.mutate   
1  gds.allShortestPaths.delta.mutate.estimate   
2            gds.allShortestPaths.delta.stats   
3   gds.allShortestPaths.delta.stats.estimate   
4           gds.allShortestPaths.delta.stream   

                                         description  \
0  The Delta Stepping shortest path algorithm com...   
1  Returns an estimation of the memory consumptio...   
2  The Delta Stepping shortest path algorithm com...   
3  Returns an estimation of the memory consumptio...   
4  The Delta Stepping shortest path algorithm com...   

                                           signature       type  
0  gds.allShortestPaths.delta.mutate(graphName ::...  procedure  
1  gds.allShortestPaths.delta.mutate.estimate(gra...  procedure  
2  gds.allShortestPaths.delta.stats(graphName :: ...  procedure  
3  gds.allShortestPaths.delta.stats.estimate(grap...  procedu

## Graph Projections

In [12]:
# Cleanup existing graph projection if it exists
if gds.graph.exists("foodweb").exists:
    gds.graph.drop("foodweb")

In [14]:
# First projection for directed analyses (degree, betweenness, closeness, communities)
G, proj_result = gds.graph.project(
    "foodweb",
    {
        "Species": {
            "properties": []
        }
    },
    {
        "eaten_by": {
            "orientation": "NATURAL",   
            "properties": []
        }
    }
)


In [15]:
print(f"Projected {G.node_count()} nodes and {G.relationship_count()} relationships for directed analyses.\n")

Projected 1876 nodes and 1658 relationships for directed analyses.



## Phase 1: Analysis



In [16]:
props_df = gds.run_cypher("""
MATCH ()-[r:eaten_by]->()
UNWIND keys(r) AS k
RETURN k AS property, count(*) AS freq
ORDER BY freq DESC
""")
print(props_df)

                    property  freq
0   predator_scientific_name  1658
1       prey_scientific_name  1658
2                observed_on  1658
3                  time_zone  1658
4                  image_url  1658
5                        url  1658
6         captive_cultivated  1658
7            prey_agreements  1658
8        predator_agreements  1658
9                   latitude  1657
10                 longitude  1657
11               place_state  1655
12             place_country  1655
13                     place  1652
14              place_county  1638
15       positional_accuracy  1401
16           type_of_feeding  1142
17                place_town   121
